In [ ]:
import multiprocessing 

import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

!pip install git+https://github.com/google/qkeras

In [ ]:
from tensorflow.keras.datasets import cifar10

def get_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    #x_train, x_test = tf.image.rgb_to_grayscale(x_train).numpy(), tf.image.rgb_to_grayscale(x_test).numpy()
    #x_train = np.average(x_train, axis=3, weights=[0.2125, 0.7154, 0.0721]) 
    #x_test = np.average(x_test, axis=3, weights=[0.2125, 0.7154, 0.0721])
    #cv2_imshow(x_train[4])
    x_train = x_train.reshape(x_train.shape + (1,)).astype("float32")
    x_test = x_test.reshape(x_test.shape + (1,)).astype("float32")

    print (np.max(x_train), np.max(x_test))
    x_train /= 256.0
    x_test /= 256.0

    x_mean = np.mean(x_train, axis=0)

    x_train -= x_mean
    x_test -= x_mean

    nb_classes = np.max(y_train)+1
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)

    #quantizer = quantized_bits(17, 1)
    #x_train = quantizer(x_train)
    #x_test = quantizer(x_test)

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = get_data()
print (x_train.shape)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
from qkeras import *
import qkeras
from qkeras.utils import *

def CreateQModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)

    x = QConv2D(32, (3, 3),
                kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
                bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
                name='c1')(x)
    x = QActivation('quantized_relu(17, {})'.format(intBits))(x)
    x = MaxPooling2D((2,2))(x)

    x = QConv2D(64, (3, 3),
                kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
                bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
                name="c2")(x)
    x = QActivation('quantized_relu(17, {})'.format(intBits))(x)
    x = MaxPooling2D((2,2))(x)

    x = QConv2D(64, (3, 3),
                kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
                bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
                name='c3')(x)
    x = QActivation('quantized_relu(17, {})'.format(intBits))(x)

    x = Flatten(name="flatten")(x)

    x = QDense(64,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense3")(x)
    x = QActivation('quantized_relu(17, {})'.format(intBits))(x)

    x = QDense(nb_classes,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense4")(x)
    x = Activation("softmax", name="softmax")(x)


    model = Model(inputs=x_in, outputs=x)
    
    return model

from tensorflow.keras.optimizers import Adam
model = CreateQModel(x_train.shape[1:], y_train.shape[-1], 1)
model.compile(
    loss="categorical_crossentropy",
    #loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(0.1),
    #optimizer='sgd',
    metrics=["accuracy"])
history = model.fit(x_train, y_train, epochs=10, batch_size=256, validation_data=(x_test[:5000], y_test[:5000]), verbose=True) # callbacks=[callback])
model_save_quantized_weights(model)
print ("Done")



# Tensorflow Colab file with some modifications

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
intBits = 1
precision = 17

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
#model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c1'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c2'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c3'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
#model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        name='d1'))
#model.add(QActivation('quantized_relu({}, {})'.format(precision, intBits)))
model.add(layers.Dense(10))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images[:5000], test_labels[:5000]))

In [ ]:
import qkeras
from qkeras import *
from qkeras.utils import *

In [ ]:
model_save_quantized_weights(model)
print ("Done")

In [ ]:
model.evaluate(test_images[5000:], test_labels[5000:])

In [ ]:
### Evaluation in a loop
intBits = 1
histories = []
mymodels = []
for precision in [8, 12, 16, 24, 32]:
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c1'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c2'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c3'))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(QActivation('quantized_relu({}, {})'.format(precision, intBits)))
  model.add(layers.Dense(10))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  history = model.fit(train_images, train_labels, epochs= 5 + int(3*((precision/8) - 1)), 
                    validation_data=(test_images[:5000], test_labels[:5000]), verbose=False)
  
  model_save_quantized_weights(model)
  print ("Done")

  model.evaluate(test_images[5000:], test_labels[5000:])

  mymodels.append(model)
  histories.append(history)